In [1]:
import tensorflow as tf

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import math

In [3]:
def scale_array(array, scale_x, scale_y):
    height, width = array.shape
    
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)
    
    scaled_array = np.zeros((new_height, new_width), dtype=array.dtype)
    
    for i in range(new_height):
        for j in range(new_width):
            original_i = int(i / scale_y)
            original_j = int(j / scale_x)
            
            if original_i >= height or original_j >= width:
                scaled_array[i, j] = 0
            else:
                scaled_array[i, j] = array[original_i, original_j]
    
    return scaled_array

In [4]:
def ElevationToImg(elevation, width, height):
    elevation_df = pd.DataFrame(elevation)
    
    max_x = max(elevation_df["x"]) + 1
    max_y = max(elevation_df["y"]) + 1
    
    array = np.zeros((max_y, max_x), dtype=int)
    
    for index, data in elevation_df.iterrows():
        x = data["x"]
        y = data["y"]
        elevation_val = data["elevation"]
        array[y, x] = elevation_val
        
    if max_x <= width and max_y <= height:
        scaleX = width / max_x
        scaleY = height / max_y
        resized_array = scale_array(array, scaleX, scaleY)
    else:
        resized_array = cv2.resize(array, (width, height), interpolation=cv2.INTER_AREA)

    return resized_array

In [5]:
def ImgDivider(image, size=32):
    piece_width = size
    piece_height = size

    height, width = image.shape

    pieces = []
    for y in range(0, height, piece_height):
        for x in range(0, width, piece_width):
            # Parça boyutlarını belirle
            piece_right = min(x + piece_width, width)
            piece_bottom = min(y + piece_height, height)
            piece_width_actual = piece_right - x
            piece_height_actual = piece_bottom - y
            
            # Parçayı oluştur ve padding yap
            piece = np.zeros((piece_height, piece_width), dtype=np.uint8)
            piece[:piece_height_actual, :piece_width_actual] = image[y:piece_bottom, x:piece_right]
            pieces.append(piece)

    return pieces


In [19]:
class AI:
    def __init__(self, model_path):
        self.model = tf.keras.models.load_model(model_path)
    
    def FetchData(self, lat, lon, lat2, lon2):
        #fake:
        Where = "izmirkuzey"
        imagesF = "AllImages"
        
        for dosya_adi in os.listdir(imagesF):
            if dosya_adi.endswith(".png"):
                parcalar = dosya_adi.split("_")
                if len(parcalar) > 1 and (parcalar[1].startswith(Where) or parcalar[0].startswith(Where)):
                    dosya_yolu = os.path.join(imagesF, dosya_adi)
                    img = cv2.imread(dosya_yolu, cv2.IMREAD_GRAYSCALE)
                    if(parcalar[0].startswith("AgricultralMap")):
                        self.AgricultralMap = img
                    elif(parcalar[0].startswith("GreenAreaMap")):
                        self.GreenAreaMap = img
                    elif(parcalar[0].startswith("LivestockMap")):
                        self.LivestockMap = img
                    elif(parcalar[0].startswith("MinesMap")):
                        self.MinesMap = img
                    elif(parcalar[0].startswith("RegionsMap")):
                        self.RegionsMap = img
                    elif(parcalar[1].startswith("ElevationMatrix") or parcalar[0].startswith("ElevationMatrix")):
                        self.Elevation = img
                        
    def GetDatas(self):
        return {
            "AgricultralMap" : self.AgricultralMap,
            "GreenAreaMap" : self.GreenAreaMap,
            "LivestockMap" : self.LivestockMap,
            "MinesMap" : self.MinesMap,
            "RegionsMap" : self.RegionsMap,
            "Elevation" : self.Elevation,
        }
    
    def Normalize(self):
        normalized_maps = np.copy(self.Elevation)
        normalized_maps[normalized_maps > 1000] = 1000
        normalized_maps = normalized_maps / 1000.0
        normalized_maps[normalized_maps == 1] = 1

        self.Elevation = normalized_maps
    
    def PrepareDataToPredict(self):
        self.Images = {
            "RegionsMaps" : [],
            "MinesMaps" : [],
            "GreenAreaMaps" : [],
            "LivestockMaps" : [],
            "AgricultralMaps" : [],
            "ElevationMaps" : [],
        }
        
        self.Images["RegionsMaps"].extend(ImgDivider(self.RegionsMap))
        self.Images["MinesMaps"].extend(ImgDivider(self.MinesMap))
        self.Images["GreenAreaMaps"].extend(ImgDivider(self.GreenAreaMap))
        self.Images["LivestockMaps"].extend(ImgDivider(self.LivestockMap))
        self.Images["AgricultralMaps"].extend(ImgDivider(self.AgricultralMap))
        self.Images["ElevationMaps"].extend(ImgDivider(self.Elevation))
        
        self.inputsVals = np.stack((self.Images["AgricultralMaps"], self.Images["MinesMaps"], self.Images["GreenAreaMaps"], self.Images["LivestockMaps"], self.Images["ElevationMaps"]), axis=-1)
    
    def Predict(self):
        self.Normalize()
        self.PrepareDataToPredict()
        predictions = self.model.predict(self.inputsVals)        
        return np.around(predictions, decimals=1)

In [20]:
M = AI("NewModelWithTerrain_4.keras")

In [21]:
M.FetchData(0, 0, 1, 1)

In [22]:
datas = M.GetDatas()

In [ ]:
preds = M.Predict()

In [24]:
height, width = (M.GetDatas()["RegionsMap"]).shape[:2]
org_width = math.ceil(width / 32) + (width % 32 > 0)
org_height = math.ceil(height / 32) + (height % 32 > 0)

In [ ]:
yeni_dizi = np.zeros((org_height, org_width))

row, col = 0, 0
for deger in preds:
    yeni_dizi[row, col] = deger
    col += 1
    if col == org_width:
        col = 0
        row += 1

In [27]:
(np.min(yeni_dizi), np.max(yeni_dizi), np.mean(yeni_dizi))

(0.0, 1.0, 0.8034072188097322)